In [12]:
import requests
import pandas as pd
from io import StringIO
from datetime import datetime, timedelta
import joblib
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [13]:
# 1. 기상청 API 데이터 수집 (2023년)
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 12, 31)
date_list = [(start_date + timedelta(days=i)).strftime('%Y%m%d') for i in range((end_date - start_date).days + 1)]

weather_records = []
col_names = [f'col{i}' for i in range(56)]
url_weather = "https://apihub.kma.go.kr/api/typ01/url/kma_sfcdd3.php"

for date in date_list:
    params_weather = {
        "tm1": date,
        "tm2": date,
        "stn": "108",
        "help": "1",
        "authKey": "n7wd-Im8T368HfiJvO9-5Q"  # 실제 인증키로 교체 필요
    }
    try:
        response_weather = requests.get(url_weather, params=params_weather, timeout=10)
        response_weather.encoding = 'euc-kr'
        lines = response_weather.text.split('\n')
        lines = [line for line in lines if line and not line.startswith('#')]
        if not lines:
            continue
        csv_data = '\n'.join(lines)
        df_w = pd.read_csv(StringIO(csv_data), sep='\s+', header=None, names=col_names, engine='python')
        df_w = df_w.reset_index(drop=True)
        df_w['date'] = df_w['col0'].astype(str).str[:8]
        df_w['date_dt'] = pd.to_datetime(df_w['date'], format='%Y%m%d', errors='coerce')
        df_w = df_w[['date_dt', 'col16', 'col21', 'col2']]
        df_w.columns = ['date_dt', 'TA_AVG', 'HM_AVG', 'WS_AVG']
        weather_records.append(df_w)
    except Exception as e:
        continue

if weather_records:
    df_weather_all = pd.concat(weather_records, ignore_index=True)
else:
    df_weather_all = pd.DataFrame()

print('기상청 데이터 수집 완료, 샘플 데이터:')
print(df_weather_all.head())
print('총 수집된 날짜 수:', df_weather_all['date_dt'].nunique())

기상청 데이터 수집 완료, 샘플 데이터:
     date_dt  TA_AVG  HM_AVG  WS_AVG
0 2023-01-01    -1.4     3.3     2.7
1 2023-01-02    -4.2     2.0     2.5
2 2023-01-03    -4.8     2.0     1.8
3 2023-01-04    -2.9     2.7     1.9
4 2023-01-05    -3.3     3.1     1.6
총 수집된 날짜 수: 365


In [14]:
# 2. 산불위험예보 목록정보 불러오기 및 전처리
df_risk = pd.read_csv('산림청 국립산림과학원_대형산불위험예보목록정보_20250430.csv', encoding='euc-kr')
df_risk = df_risk.rename(columns={
    '예보일시': 'date',
    '시도명': 'province',
    '시군구명': 'city',
    '실효습도': 'effective_humidity',
    '풍속': 'wind_speed',
    '등급': 'risk_grade'
})
df_risk['date_dt'] = pd.to_datetime(df_risk['date'], errors='coerce').dt.date
df_risk['effective_humidity'] = pd.to_numeric(df_risk['effective_humidity'], errors='coerce')
df_risk['wind_speed'] = pd.to_numeric(df_risk['wind_speed'], errors='coerce')

if not df_weather_all.empty:
    df_weather_all['date_dt'] = pd.to_datetime(df_weather_all['date_dt']).dt.date
    risk_weather = pd.merge(df_risk, df_weather_all, on='date_dt', how='inner')
else:
    risk_weather = pd.DataFrame()


In [15]:
# 3. 산불상황관제시스템 통계데이터 불러오기 및 전처리
df_fire = pd.read_csv('sanbul.csv', encoding='cp949')
df_fire = df_fire.rename(columns={
    '발생일시_년': 'year',
    '발생일시_월': 'month',
    '발생일시_일': 'day',
    '발생장소_시도': 'province',
    '발생장소_시군구': 'city'
})
df_fire['date_dt'] = pd.to_datetime(
    df_fire['year'].astype(str) + '-' +
    df_fire['month'].astype(str).str.zfill(2) + '-' +
    df_fire['day'].astype(str).str.zfill(2),
    errors='coerce'
).dt.date

fire_group = df_fire.groupby(['date_dt', 'province', 'city']).size().reset_index(name='fire_count')
fire_group['fire_occurred'] = (fire_group['fire_count'] > 0).astype(int)


In [16]:
# 4. 데이터 병합 및 결측치 처리
final_df_2023 = pd.merge(
    risk_weather,
    fire_group,
    on=['date_dt', 'province', 'city'],
    how='left'
)
final_df_2023['fire_occurred'] = final_df_2023['fire_occurred'].fillna(0).astype(int)
final_df_2023['fire_count'] = final_df_2023['fire_count'].fillna(0).astype(int)

features = ['TA_AVG', 'HM_AVG', 'WS_AVG', 'effective_humidity', 'wind_speed']
final_df_2023 = final_df_2023.dropna(subset=features)

print('2023년 데이터 전처리 완료, 샘플 데이터:')
print(final_df_2023.head())
print('총 데이터 수:', len(final_df_2023))

2023년 데이터 전처리 완료, 샘플 데이터:
               date province city 읍면동명  effective_humidity  wind_speed  \
0  2023-01-02 00:00       강원   양양  손양면                30.4         7.1   
1  2023-01-02 03:00       부산  금정구  노포동                32.5         8.4   
2  2023-01-02 03:00       전남   여수   국동                34.4         8.1   
3  2023-01-02 03:00       전남   여수  만흥동                34.4         8.8   
4  2023-01-02 03:00       전남   여수  봉계동                34.4         8.0   

  risk_grade     date_dt  TA_AVG  HM_AVG  WS_AVG  fire_count  fire_occurred  
0        주의보  2023-01-02    -4.2     2.0     2.5           0              0  
1        주의보  2023-01-02    -4.2     2.0     2.5           0              0  
2        주의보  2023-01-02    -4.2     2.0     2.5           0              0  
3        주의보  2023-01-02    -4.2     2.0     2.5           0              0  
4        주의보  2023-01-02    -4.2     2.0     2.5           0              0  
총 데이터 수: 3813


In [17]:
# 5. 저장된 모델 불러오기 및 성능 평가
X_2023 = final_df_2023[features].astype(float)
y_2023 = final_df_2023['fire_occurred']

model = joblib.load('xgb_fire_model_smote.pkl')

preds_2023 = model.predict(X_2023)

print("2023년 데이터 정확도:", accuracy_score(y_2023, preds_2023))
print(classification_report(y_2023, preds_2023))
print(confusion_matrix(y_2023, preds_2023))

2023년 데이터 정확도: 0.9737739312877
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      3608
           1       0.69      0.95      0.80       205

    accuracy                           0.97      3813
   macro avg       0.84      0.96      0.89      3813
weighted avg       0.98      0.97      0.98      3813

[[3519   89]
 [  11  194]]


In [18]:
# 6. 예측 결과 표로 확인
result_df_2023 = X_2023.copy()
result_df_2023['actual_fire_occurred'] = y_2023
result_df_2023['predicted_fire_occurred'] = preds_2023
print(result_df_2023.head())

   TA_AVG  HM_AVG  WS_AVG  effective_humidity  wind_speed  \
0    -4.2     2.0     2.5                30.4         7.1   
1    -4.2     2.0     2.5                32.5         8.4   
2    -4.2     2.0     2.5                34.4         8.1   
3    -4.2     2.0     2.5                34.4         8.8   
4    -4.2     2.0     2.5                34.4         8.0   

   actual_fire_occurred  predicted_fire_occurred  
0                     0                        0  
1                     0                        0  
2                     0                        0  
3                     0                        0  
4                     0                        0  
